In [47]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import re
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [48]:
classes = os.listdir('./data/input_images')
classes

['paper', 'cardboard', 'plastic', 'metal', 'trash', 'glass']

In [49]:
def get_picture_per_classes(classes):
    return [os.listdir('./data/input_images/'+ item) for item in classes]

In [50]:
images = get_picture_per_classes(classes)

In [51]:
def get_image_name(item):
    return item.split('.')[0]

In [52]:
images_name = [get_image_name(item) for image in images for item in image]

In [53]:
def get_classes_from_images(item):
    num = re.findall('\d+', item)
    return item.split(num[0])[0]

In [54]:
image_classes = [get_classes_from_images(item) for item in images_name]

In [55]:
new_dict = {'Images_name':images_name, 'Image_class': image_classes}

In [56]:
p = pd.DataFrame(new_dict, index=images_name, columns=['Image_class'])
p.head()

,Image_class
paper85,paper
paper503,paper
paper75,paper
paper482,paper
paper300,paper


In [57]:
new_p = pd.get_dummies(p['Image_class'])
new_p.head()
new_p.to_csv(r'./images_dummies.csv')

In [58]:
# paths:
DATASET = 'images_dummies.csv'
ROOT_IMG = './data/input_images/'
ROOT_IMG_DAUGHTERS = classes
ROOT_MODELS = 'Models/'
# img size:
ROW_IMAGES = 384
COL_IMAGES = 512
IMG_COLORS = 3
input_shape = (ROW_IMAGES, COL_IMAGES, IMG_COLORS)
# num of pictures:
PIC_NUM = 2527
# train-test parameters:
TRAIN_RATIO = 0.2
RANDOM_STATE = 42
# fit parameters:
BATCH_SIZE = 20
EPOCHS =10

COLUMNS = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
num_classes = len(COLUMNS)

In [59]:
new_p['image'] = new_p.index
new_p.head()

,cardboard,glass,metal,paper,plastic,trash,image
paper85,0,0,0,1,0,0,paper85
paper503,0,0,0,1,0,0,paper503
paper75,0,0,0,1,0,0,paper75
paper482,0,0,0,1,0,0,paper482
paper300,0,0,0,1,0,0,paper300


In [60]:
def get_image(item):
    return item + '.jpg'

In [61]:
def get_parental_path(item):
    parental = get_classes_from_images(item)
    return parental + '/'+ get_image(item)

In [62]:
def get_array(a, b, c, d, e, f): 
    return [a, b, c, d, e, f]

In [63]:
images=[]
wastes=[]
for row, index_row in new_p.iterrows():
    #tomo la ruta de la imagen
    image_root = get_parental_path(row)
    #leo la imagen
    read_image = cv2.imread(ROOT_IMG + image_root)
    #gardo la imagen en images
    gray_image = cv2.cvtColor(read_image, cv2.COLOR_BGR2GRAY)
    resized_img = cv2.resize(gray_image, None, fx=0.5, fy=0.5)
    expanded_dim = np.expand_dims(resized_img, axis=2)
    #float_image = gray_image.astype('float32')
    #float_image /= 255
    images.append(expanded_dim)
    
    #creo un array de 0s y 1s para saber el tipo de residuo presente en la imagen
    wastes.append(get_array(new_p['cardboard'][row], new_p['glass'][row], new_p['metal'][row], 
            new_p['paper'][row], new_p['plastic'][row], new_p['trash'][row]))
    # lo guardo en wastes
    #wastes.append()
images
wastes

[[0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1,

In [64]:
x = np.array(images)
y = np.array(wastes)

In [65]:
x[0].shape

(192, 256, 1)

def get_matrix():
    result = []
    columns = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
    for col in columns:
        result.append(new_p[col])
    return result

def image_to_array(image,inx,iny):
    # Converts image to an inx * iny size, change color to greyscale and flatten it into 1D ndarray
    image = cv2.resize(image,(inx,iny)) 
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
    image = np.reshape(image,(inx,iny))
    return np.ndarray.flatten(image)

def transform_data(item):
    

COLUMNS = new_p.columns
data = pd.DataFrame(new_p,columns=['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash', 'image'])
data.reset_index(drop=True)
data.to_csv('data.csv')
data.head()

type(df.image[0])
imagen = cv2.imread(ROOT_IMG+'paper/paper1.jpg')
for lista in imagen:
    for pixel in lista:
        print(pixel/255)

def replace_value(item):
    return item.replace('\n', ',')

imagen= cv2.imread('./data/input_images/paper/paper7.jpg')
plt.imshow(Image.fromarray(imagen.squeeze()))


In [66]:
import sklearn
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=TRAIN_RATIO, random_state=RANDOM_STATE)

In [68]:
x.shape
y.shape

(2527, 6)

In [69]:
# Prepare data to feed the NN
num_classes = 6

"""
if K.image_data_format() == 'channels_first':
    input_shape = [1, ROW_IMAGES, COL_IMAGES]
    X_train = X_train.reshape(X_train.shape[0], 1, ROW_IMAGES, COL_IMAGES)
    X_test = X_test.reshape(X_test.shape[0], 1, ROW_IMAGES, COL_IMAGES)
    
else:
    print('hola')
    print(X_train.shape)
    input_shape = [ROW_IMAGES, COL_IMAGES, 1]
    X_train.reshape(X_train.shape[0], input_shape[0], input_shape[1], input_shape[2])
    X_test.reshape(X_test.shape[0], input_shape[0], input_shape[1], input_shape[2])"""# Ask keras which format to use depending on used backend and arrange data as expected

    
# Incoming data is in uint8. Cast the input data images to be floats in range [0.0-1.0]
#normalize = lambda pixel: pixel/255 
#x_train = normalize(X_train)

X_train = X_train.astype('float32')
#X_test = normalize(X_test)
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
# convert class vectors to binary class matrices
"""y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)"""
X_train

x_train shape: (2021, 192, 256, 1)
2021 train samples
506 test samples


array([[[[1.        ],
         [0.99607843],
         [0.99215686],
         ...,
         [0.8235294 ],
         [0.81960785],
         [0.8156863 ]],

        [[1.        ],
         [1.        ],
         [0.99607843],
         ...,
         [0.8235294 ],
         [0.81960785],
         [0.8156863 ]],

        [[1.        ],
         [1.        ],
         [0.99607843],
         ...,
         [0.8235294 ],
         [0.81960785],
         [0.8156863 ]],

        ...,

        [[0.5568628 ],
         [0.49411765],
         [0.49019608],
         ...,
         [0.18039216],
         [0.16470589],
         [0.16862746]],

        [[0.44705883],
         [0.49019608],
         [0.44705883],
         ...,
         [0.16862746],
         [0.16078432],
         [0.16078432]],

        [[0.46666667],
         [0.5529412 ],
         [0.38431373],
         ...,
         [0.16078432],
         [0.17254902],
         [0.17254902]]],


       [[[0.67058825],
         [0.67058825],
         [0.66

In [43]:
y_train

array([[[1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.]],

       [[1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.]],

       [[0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.]],

       ...,

       [[1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.]],

       [[1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.

X_train = X_train.astype('float32')
#X_test = normalize(X_test)
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

In [70]:
## This is the neural network proposed architecture
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu', input_shape=(X_train.shape[1:])))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer=keras.optimizers.Adadelta(),
             metrics=['accuracy'])

In [71]:
batch_size = 20
epochs = 10
model.fit(X_train, y_train,
         batch_size=batch_size,
         epochs=epochs,
         verbose=1,
         validation_data=(X_test,  y_test))

Train on 2021 samples, validate on 506 samples
Epoch 1/10
2021/2021 [==============================] - 262s 129ms/step - loss: 3.0044 - accuracy: 0.2598 - val_loss: 1.7955 - val_accuracy: 0.2194
Epoch 2/10
2021/2021 [==============================] - 254s 125ms/step - loss: 1.6064 - accuracy: 0.3295 - val_loss: 1.6659 - val_accuracy: 0.3083
Epoch 3/10
2021/2021 [==============================] - 248s 123ms/step - loss: 1.4731 - accuracy: 0.4196 - val_loss: 1.4710 - val_accuracy: 0.4229
Epoch 4/10
2021/2021 [==============================] - 252s 125ms/step - loss: 1.2500 - accuracy: 0.5364 - val_loss: 1.4875 - val_accuracy: 0.4664
Epoch 5/10
2021/2021 [==============================] - 253s 125ms/step - loss: 1.0356 - accuracy: 0.6304 - val_loss: 1.3984 - val_accuracy: 0.4585
Epoch 6/10
2021/2021 [==============================] - 249s 123ms/step - loss: 0.7367 - accuracy: 0.7427 - val_loss: 1.8646 - val_accuracy: 0.3874
Epoch 7/10
2021/2021 [==============================] - 249s 123m

In [72]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])



Test loss: 1.771059890509594
Test accuracy: 0.4881422817707062


In [78]:
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
img = X_test[30]

pred = model.predict(img)
print("Probs -> Cardboard:{0:.5f} Glass:{1:.5f} Metal:{2:.5f} Paper:{3:.5f} Plastic:{4:.5f} Trash:{5:.5f}".format(pred[0],pred[1]))
plt.imshow(Image.fromarray(img.squeeze()*255),'gray')

ValueError: Error when checking input: expected conv2d_13_input to have 4 dimensions, but got array with shape (192, 256, 1)